In [2]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf
from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [5]:
dataset_name = "plane-dataset" # The name of the V51 dataset that will be used
training_name = "simple_model"

In [6]:
# The different directories and filenames to use
train_record_fname = "/tf/dataset-export/" + training_name + "/train/tf.records"
val_record_fname = "/tf/dataset-export/" + training_name + "/val/tf.records"

In [10]:
raw_image_dataset = tf.data.TFRecordDataset(train_record_fname)
image_feature_description = {
    # Image dimensions
    "image/height": tf.io.FixedLenFeature([], tf.int64),
    "image/width": tf.io.FixedLenFeature([], tf.int64),

    # Image filename is used for both of these when writing
    "image/filename": tf.io.FixedLenFeature([], tf.string),
    "image/source_id": tf.io.FixedLenFeature([], tf.string),

    # Encoded image bytes
    "image/encoded": tf.io.FixedLenFeature([], tf.string),

    # Image format, either `jpeg` or `png`
    "image/format": tf.io.FixedLenFeature([], tf.string),

    # Normalized bounding box coordinates in `[0, 1]`
    "image/object/bbox/xmin": tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    "image/object/bbox/xmax": tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    "image/object/bbox/ymin": tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    "image/object/bbox/ymax": tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

    # Class label string
    "image/object/class/text": tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),

    # Integer class ID
    "image/object/class/label": tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True)
}
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)



In [11]:
parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {image/encoded: (), image/filename: (), image/format: (), image/height: (), image/object/bbox/xmax: (None,), image/object/bbox/xmin: (None,), image/object/bbox/ymax: (None,), image/object/bbox/ymin: (None,), image/object/class/label: (None,), image/object/class/text: (None,), image/source_id: (), image/width: ()}, types: {image/encoded: tf.string, image/filename: tf.string, image/format: tf.string, image/height: tf.int64, image/object/bbox/xmax: tf.float32, image/object/bbox/xmin: tf.float32, image/object/bbox/ymax: tf.float32, image/object/bbox/ymin: tf.float32, image/object/class/label: tf.int64, image/object/class/text: tf.string, image/source_id: tf.string, image/width: tf.int64}>

In [15]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image/encoded'].numpy()
  display(Image(data=image_raw))

TypeError: 'module' object is not callable